<a href="https://colab.research.google.com/github/kircherlab/MPRAsnakeflow_tutorial/blob/development/tutorial_association.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MPRAsnakeflow association tutorial

In [ ]:
import matplotlib.pyplot as plt
import os
import cv2
from google.colab.patches import cv2_imshow
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
gdrive_dir = "/content/gdrive/MyDrive/MPRA-IGVF-Workshop-2024/Tutorials/T2_MPRA_stats/MPRAsnakeflow_data"

First, let's take a look at the structure of our input files:

!ls -R "{gdrive_dir}"

In [ ]:
%%bash
mkdir -p /content/work
cd /content/work
ln -s /content/gdrive/MyDrive/MPRA-IGVF-Workshop-2024/Tutorials/T2_MPRA_stats/MPRAsnakeflow_data/assignment_config.yaml
ln -s /content/gdrive/MyDrive/MPRA-IGVF-Workshop-2024/Tutorials/T2_MPRA_stats/MPRAsnakeflow_data/design
ln -s /content/gdrive/MyDrive/MPRA-IGVF-Workshop-2024/Tutorials/T2_MPRA_stats/MPRAsnakeflow_data/assignment

## Pre-requirements

### Docker
Docker using udocker in colab from github.com/drengskapur/docker-in-colab.

In [ ]:
# Copyright 2024 Drengskapur
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#
# @title {display-mode:"form"}
# @markdown <br/><br/><center><img src="https://cdn.jsdelivr.net/gh/drengskapur/docker-in-colab/assets/docker.svg" height="150"><img src="https://cdn.jsdelivr.net/gh/drengskapur/docker-in-colab/assets/colab.svg" height="150"></center><br/>
# @markdown <center><h1>Docker in Colab</h1></center><center>github.com/drengskapur/docker-in-colab<br/><br/><br/><b>udocker("run hello-world")</b></center><br/>
def udocker_init():
    import os
    if not os.path.exists("/home/user"):
        !pip install udocker > /dev/null
        !udocker --allow-root install > /dev/null
        !useradd -m user > /dev/null
    print(f'Docker-in-Colab 1.1.0\n')
    print(f'Usage:     udocker("--help")')
    print(f'Examples:  https://github.com/indigo-dc/udocker?tab=readme-ov-file#examples')

    def execute(command: str):
        user_prompt = "\033[1;32muser@pc\033[0m"
        print(f"{user_prompt}$ udocker {command}")
        !su - user -c "udocker $command"

    return execute

udocker = udocker_init()

Docker-in-Colab 1.1.0

Usage:     udocker("--help")
Examples:  https://github.com/indigo-dc/udocker?tab=readme-ov-file#examples


### snakemake

snakemake v8.16.0 using the official docker image.

In [ ]:
udocker("pull visze/snakemake_mprasnakeflow:0.1.0")
udocker("create --name=mprasnakeflow visze/snakemake_mprasnakeflow:0.1.0")

user@pc$ udocker pull visze/snakemake_mprasnakeflow:0.1.0
Info: downloading layer sha256:ca895cfad24f83bcf15712d53a79a7ca7afd230c1044c3e46e59b2a6ee8fd4e5
Info: downloading layer sha256:6a1ace6fb4a7b40b0f1019fa1a3225d2970f62518b41c272b3b462acc0e72d4f
Info: downloading layer sha256:030e57b99b3b4f88a0d85eea59fcef5b8d7f7301a9992c88f9cab73191ad554f
Info: downloading layer sha256:d7e123fc54cd9ae8ee4d671f0f7b134486532e5f07de84d09cb3e4a521f2b848
Info: downloading layer sha256:2241bf831ea24afc27d2f4974c716a3e8bd7dee9f213c89fb93e8bb25d4f14cd
Info: downloading layer sha256:c6e539d6d31625c3c6ec4170ceeaa960c9779c60d4839e3ef0d5cd5251331943
Info: downloading layer sha256:e71e00cb9af4e3c09ee580b9c031c586c30fa46d4934445a20c36e070d37023a
Info: downloading layer sha256:fee224ae51e3aff735790e41da3a59005e455f38f97062961a57e193ff19de11
Info: downloading layer sha256:5466efd28f652843211c2f22d358d79cd7fc127dcaae1d2de5a5cbc09e9bfd24
Info: downloading layer sha256:0ebeb0a01cdfb95005934857d1270693f20340315f7f729

Try to run `snakemake --version` in docker:

In [ ]:
udocker("run mprasnakeflow snakemake --version")

user@pc$ udocker run visze/snakemake_mprasnakeflow:0.1.0 snakemake --version
 
 ****************************************************************************** 
 *                                                                            * 
 *               STARTING 5840c75d-aabb-3605-ad0f-2eccc7bae286                * 
 *                                                                            * 
 ****************************************************************************** 
 executing: _entrypoint.sh
8.16.0+0.g838b2fce.dirty


## Assignment input data

In [ ]:
%%bash
mkdir -p assoc_basic/data
cd assoc_basic/data
curl -s ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM4237nnn/GSM4237954/suppl/GSM4237954_9MPRA_elements.fa.gz | \
zcat - |awk '{ count+=1; if (count == 1) { print } else { print substr($1,1,171)}; if (count == 2) { count=0 } }' | awk '{gsub(/[\]\[]/,"_")} $0' > design.fa

wget -q -O SRR10800986_1.fastq.gz https://kircherlab.bihealth.org/download/ali/SRR10800986_1.200K.fastq.gz
wget -q -O SRR10800986_2.fastq.gz https://kircherlab.bihealth.org/download/ali/SRR10800986_2.200K.fastq.gz
wget -q -O SRR10800986_3.fastq.gz https://kircherlab.bihealth.org/download/ali/SRR10800986_3.200K.fastq.gz

#### Complete data

Install `sra-tools` for fetchicng assignment data from SRA.

In [ ]:
!mamba install -y -qq -c bioconda -c conda-forge sra-tools

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


In [ ]:
%%bash
cd assoc_basic/data
prefetch SRR10800986
fastq-dump --gzip --split-files SRR10800986

Process is interrupted.


### MPRAsnakeflow workflow

In [ ]:
%%bash
cd /content
git clone https://github.com/kircherlab/MPRAsnakeflow.git
cd MPRAsnakeflow && git checkout development

Branch 'development' set up to track remote branch 'development' from 'origin'.


Cloning into 'MPRAsnakeflow'...
Switched to a new branch 'development'


In [ ]:
!mkdir -p /data
!mkdir -p /content/apptainer
!ln -s /content/MPRAsnakeflow /data/MPRAsnakeflow

## Run MPRAsnakeflow

### Assignment data

/content/MPRAsnakeflow/resources/assoc_basic/config.yml

First a dry run `-n` just showing the overview of excecuted rules `--quiet rules` using one core `-c 1`.

We have to let snakemake know the main `Snakefile` of MPRAsnakeflow using `--snakefile /data/MPRAsnakeflow/workflow/Snakefile`.

Also the workflow with input files in configured in a YAML file and we give it as input with `--configfile /data/MPRAsnakeflow/resources/assoc_basic/config.yml`

 Please note that we map `/content/MPRAsnakeflow` to `/data/MPRAsnakeflow/` inside the snakemake container.



In [ ]:
!chmod og+w assoc_basic
!ls -lh

total 16K
drwxr-xr-x  2 root root 4.0K Jul 31 08:25 apptainer
drwxrwxrwx  3 root root 4.0K Jul 31 08:12 assoc_basic
drwxr-xr-x 10 root root 4.0K Jul 31 08:30 MPRAsnakeflow
drwxr-xr-x  1 root root 4.0K Jul 29 13:22 sample_data


In [ ]:
udocker("run \
  -v=${PWD}/assoc_basic:/data/run \
  --workdir /data/run \
  visze/snakemake_mprasnakeflow:0.1.0 \
  snakemake --sdm conda --conda-prefix /data/conda_envs \
  --snakefile /data/MPRAsnakeflow/workflow/Snakefile \
  --configfile /data/MPRAsnakeflow/resources/assoc_basic/config.yml \
  -c 1 -n --quiet rules")

user@pc$ udocker run   -v=/content/assoc_basic:/data/run   --workdir /data/run   visze/snakemake_mprasnakeflow:0.1.0   snakemake --sdm conda --conda-prefix /data/conda_envs   --snakefile /data/MPRAsnakeflow/workflow/Snakefile   --configfile /data/MPRAsnakeflow/resources/assoc_basic/config.yml   -c 1 -n --quiet rules
 
 ****************************************************************************** 
 *                                                                            * 
 *               STARTING 22247a11-3131-393c-b3b9-821915dd57a0                * 
 *                                                                            * 
 ****************************************************************************** 
 executing: _entrypoint.sh
Building DAG of jobs...
Your conda installation is not configured to use strict channel priorities. This is however crucial for having robust and correct environments (for details, see https://conda-forge.org/docs/user/tipsandtricks.html). Please c

In [ ]:
udocker("run \
  -v=${PWD}/assoc_basic:/data/run \
  --workdir /data/run \
  visze/snakemake_mprasnakeflow:0.1.0 \
  snakemake --sdm conda --conda-prefix /data/conda_envs \
  --snakefile /data/MPRAsnakeflow/workflow/Snakefile \
  --configfile /data/MPRAsnakeflow/resources/assoc_basic/config.yml \
  -c 1")

user@pc$ udocker run   -v=${PWD}/assoc_basic:/data/run   --workdir /data/run   visze/snakemake_mprasnakeflow:0.1.0   snakemake --sdm conda --conda-prefix /data/conda_envs   --snakefile /data/MPRAsnakeflow/workflow/Snakefile   --configfile /data/MPRAsnakeflow/resources/assoc_basic/config.yml   -c 1
 
 ****************************************************************************** 
 *                                                                            * 
 *               STARTING f4f85828-4751-387c-9256-fd5b4b85bfcb                * 
 *                                                                            * 
 ****************************************************************************** 
 executing: _entrypoint.sh
Assuming unrestricted shared filesystem usage.
Building DAG of jobs...
Your conda installation is not configured to use strict channel priorities. This is however crucial for having robust and correct environments (for details, see https://conda-forge.org/docs/user/t

In [ ]:
!apt install -y software-properties-common
!add-apt-repository -y ppa:apptainer/ppa
!apt update
!apt install -y apptainer

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
software-properties-common is already the newest version (0.99.22.9).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Repository: 'deb https://ppa.launchpadcontent.net/apptainer/ppa/ubuntu/ jammy main'
Description:
This PPA contains stable releases of Apptainer.

Documentation https://apptainer.org/docs/
Support https://apptainer.org/help/
Bug report https://github.com/apptainer/apptainer/issues
More info: https://launchpad.net/~apptainer/+archive/ubuntu/ppa
Adding repository.
Adding deb entry to /etc/apt/sources.list.d/apptainer-ubuntu-ppa-jammy.list
Adding disabled deb-src entry to /etc/apt/sources.list.d/apptainer-ubuntu-ppa-jammy.list
Adding key to /etc/apt/trusted.gpg.d/apptainer-ubuntu-ppa.gpg with fingerprint F6B0F5193D4F3301EF491FF0AFE36534FC6218AE
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ub

In [ ]:
!apt install -y apptainer-suid

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  apptainer-suid
0 upgraded, 1 newly installed, 0 to remove and 47 not upgraded.
Need to get 5,915 kB of archives.
After this operation, 18.5 MB of additional disk space will be used.
Get:1 https://ppa.launchpadcontent.net/apptainer/ppa/ubuntu jammy/main amd64 apptainer-suid amd64 1.3.2-1~ubuntu22.04.0+stable1 [5,915 kB]
Fetched 5,915 kB in 11s (518 kB/s)
Selecting previously unselected package apptainer-suid.
(Reading database ... 124865 files and directories currently installed.)
Preparing to unpack .../apptainer-suid_1.3.2-1~ubuntu22.04.0+stable1_amd64.deb ...
Unpacking apptainer-suid (1.3.2-1~ubuntu22.04.0+stable1) ...
Setting up apptainer-suid (1.3.2-1~ubuntu22.04.0+stable1) ...


In [ ]:
!apptainer build snakemake.sif docker://snakemake/snakemake:latest

Error for command "build": unknown flag: --security

Options for build command:

  -B, --bind stringArray         a user-bind path specification. spec has the format
                                 src[:dest[:opts]],where src and dest are outside and inside
                                 paths. If dest is not given,it is set equal to src. Mount
                                 options ('opts') may be specified as 'ro'(read-only) or 'rw'
                                 (read/write, which is the default).Multiple bind paths can be
                                 given by a comma separated list.
      --build-arg strings        defines variable=value to replace {{ variable }} entries in
                                 build definition file
      --build-arg-file string    specifies a file containing variable=value lines to replace
                                 '{{ variable }}' with value in build definition files
      --disable-cache            do not use cache or create cache
 

In [ ]:
!apptainer exec --security seccomp=unconfined --security systempaths=unconfined --security no-new-privileges snakemake.sif snakemake --help

FATAL:   container creation failed: mount hook function failure: mount overlay->/var/lib/apptainer/mnt/session/final error: while mounting overlay: can't mount overlay filesystem to /var/lib/apptainer/mnt/session/final: while setting effective capabilities: CAP_DAC_READ_SEARCH is not in the permitted capability set


In [ ]:
!apptainer instance list

INSTANCE NAME    PID    IP    IMAGE


In [ ]:
!curl -L -O "https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-$(uname)-$(uname -m).sh"
!bash Miniforge3-$(uname)-$(uname -m).sh -b

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 78.9M  100 78.9M    0     0  70.7M      0  0:00:01  0:00:01 --:--:--  231M
PREFIX=/root/miniforge3
Unpacking payload ...
Extracting _libgcc_mutex-0.1-conda_forge.tar.bz2
Extracting ca-certificates-2024.2.2-hbcca054_0.conda
Extracting ld_impl_linux-64-2.40-h41732ed_0.conda
Extracting libstdcxx-ng-13.2.0-h7e041cc_5.conda
Extracting pybind11-abi-4-hd8ed1ab_3.tar.bz2
Extracting python_abi-3.10-4_cp310.conda
Extracting tzdata-2024a-h0c530f3_0.conda
Extracting libgomp-13.2.0-h807b86a_5.conda
Extracting _openmp_mutex-4.5-2_gnu.tar.bz2
Extracting libgcc-ng-13.2.0-h807b86a_5.conda
Extracting bzip2-1.0.8-hd590300_5.conda
Extracting c-ares-1.28.1-hd590300_0.conda
Extracting fmt-10.2

In [ ]:
!~/miniforge3/bin/mamba init

no change     /root/miniforge3/condabin/conda
no change     /root/miniforge3/bin/conda
no change     /root/miniforge3/bin/conda-env
no change     /root/miniforge3/bin/activate
no change     /root/miniforge3/bin/deactivate
no change     /root/miniforge3/etc/profile.d/conda.sh
no change     /root/miniforge3/etc/fish/conf.d/conda.fish
no change     /root/miniforge3/shell/condabin/Conda.psm1
no change     /root/miniforge3/shell/condabin/conda-hook.ps1
no change     /root/miniforge3/lib/python3.10/site-packages/xontrib/conda.xsh
no change     /root/miniforge3/etc/profile.d/conda.csh
no change     /root/.bashrc
No action taken.
Added mamba to /root/.bashrc

==> For changes to take effect, close and re-open your current shell. <==



In [ ]:
%%bash
__conda_setup="$('/root/miniforge3/bin/conda' 'shell.bash' 'hook' 2> /dev/null)"
if [ $? -eq 0 ]; then
    eval "$__conda_setup"
else
    if [ -f "/root/miniforge3/etc/profile.d/conda.sh" ]; then
        . "/root/miniforge3/etc/profile.d/conda.sh"
    else
        export PATH="/root/miniforge3/bin:$PATH"
    fi
fi
unset __conda_setup

if [ -f "/root/miniforge3/etc/profile.d/mamba.sh" ]; then
    . "/root/miniforge3/etc/profile.d/mamba.sh"
fi
mamba config --add channels defaults
mamba config --add channels bioconda
mamba config --add channels conda-forge
mamba config --set channel_priority strict
mamba create -y -n snakemake snakemake

conda-forge/linux-64                                        Using cache
conda-forge/noarch                                          Using cache
bioconda/linux-64                                           Using cache
bioconda/noarch                                             Using cache
Transaction

  Prefix: /root/miniforge3/envs/snakemake

  Updating specs:

   - snakemake


  Package                                   Version  Build                Channel           Size
──────────────────────────────────────────────────────────────────────────────────────────────────
  Install:
──────────────────────────────────────────────────────────────────────────────────────────────────

  + python_abi                                 3.12  4_cp312              conda-forge     Cached
  + _libgcc_mutex                               0.1  conda_forge          conda-forge     Cached
  + ld_impl_linux-64                           2.40  hf3520f5_7           conda-forge     Cached
  + ca-certificates   

In [ ]:
%%bash
__conda_setup="$('/root/miniforge3/bin/conda' 'shell.bash' 'hook' 2> /dev/null)"
if [ $? -eq 0 ]; then
    eval "$__conda_setup"
else
    if [ -f "/root/miniforge3/etc/profile.d/conda.sh" ]; then
        . "/root/miniforge3/etc/profile.d/conda.sh"
    else
        export PATH="/root/miniforge3/bin:$PATH"
    fi
fi
unset __conda_setup

if [ -f "/root/miniforge3/etc/profile.d/mamba.sh" ]; then
    . "/root/miniforge3/etc/profile.d/mamba.sh"
fi
mamba activate snakemake
snakemake --help

usage: snakemake [-h] [--dry-run] [--profile PROFILE] [--workflow-profile WORKFLOW_PROFILE]
                 [--cache [RULE ...]] [--snakefile FILE] [--cores N] [--jobs N] [--local-cores N]
                 [--resources NAME=INT [NAME=INT ...]]
                 [--set-threads RULE=THREADS [RULE=THREADS ...]] [--max-threads MAX_THREADS]
                 [--set-resources RULE:RESOURCE=VALUE [RULE:RESOURCE=VALUE ...]]
                 [--set-scatter NAME=SCATTERITEMS [NAME=SCATTERITEMS ...]]
                 [--set-resource-scopes RESOURCE=[global|local] [RESOURCE=[global|local] ...]]
                 [--default-resources [NAME=INT ...]]
                 [--preemptible-rules [PREEMPTIBLE_RULES ...]]
                 [--preemptible-retries PREEMPTIBLE_RETRIES] [--configfile FILE [FILE ...]]
                 [--config [KEY=VALUE ...]] [--envvars VARNAME [VARNAME ...]] [--directory DIR]
                 [--touch] [--keep-going]
                 [--rerun-triggers {code,input,mtime,params,soft

In [ ]:
%%bash

__conda_setup="$('/root/miniforge3/bin/conda' 'shell.bash' 'hook' 2> /dev/null)"
if [ $? -eq 0 ]; then
    eval "$__conda_setup"
else
    if [ -f "/root/miniforge3/etc/profile.d/conda.sh" ]; then
        . "/root/miniforge3/etc/profile.d/conda.sh"
    else
        export PATH="/root/miniforge3/bin:$PATH"
    fi
fi
unset __conda_setup

if [ -f "/root/miniforge3/etc/profile.d/mamba.sh" ]; then
    . "/root/miniforge3/etc/profile.d/mamba.sh"
fi

cd assoc_basic
mamba activate snakemake
snakemake --sdm apptainer --snakefile ../MPRAsnakeflow/workflow/Snakefile \
  --configfile ../MPRAsnakeflow/resources/assoc_basic/config.yml \
  -c 1 -n --quiet rules


Building DAG of jobs...
Singularity image docker://visze/mprasnakeflow:0.1.0 will be pulled.
Job stats:
job                                    count
-----------------------------------  -------
all                                        1
assignment_attach_idx                     60
assignment_bwa_ref                         1
assignment_check_design                    1
assignment_collect                         1
assignment_collectBCs                      1
assignment_fastq_split                     3
assignment_filter                          1
assignment_flagstat                        1
assignment_getBCs                         30
assignment_idx_bam                         1
assignment_mapping_bwa                    30
assignment_merge                          30
assignment_statistic_assignedCounts        1
assignment_statistic_assignment            1
assignment_statistic_totalCounts           1
total                                    164



In [ ]:
%%bash

__conda_setup="$('/root/miniforge3/bin/conda' 'shell.bash' 'hook' 2> /dev/null)"
if [ $? -eq 0 ]; then
    eval "$__conda_setup"
else
    if [ -f "/root/miniforge3/etc/profile.d/conda.sh" ]; then
        . "/root/miniforge3/etc/profile.d/conda.sh"
    else
        export PATH="/root/miniforge3/bin:$PATH"
    fi
fi
unset __conda_setup

if [ -f "/root/miniforge3/etc/profile.d/mamba.sh" ]; then
    . "/root/miniforge3/etc/profile.d/mamba.sh"
fi

cd assoc_basic
mamba activate snakemake
snakemake --sdm apptainer --snakefile ../MPRAsnakeflow/workflow/Snakefile \
  --configfile ../MPRAsnakeflow/resources/assoc_basic/config.yml \
  -c 1


Assuming unrestricted shared filesystem usage.
Building DAG of jobs...
Pulling singularity image docker://visze/mprasnakeflow:0.1.0.
Using shell: /usr/bin/bash
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Conda environments: ignored
Job stats:
job                                    count
-----------------------------------  -------
all                                        1
assignment_attach_idx                     60
assignment_bwa_ref                         1
assignment_check_design                    1
assignment_collect                         1
assignment_collectBCs                      1
assignment_fastq_split                     3
assignment_filter                          1
assignment_flagstat                        1
assignment_getBCs                         30
assignment_idx_bam                         1
assignment_mapping_bwa                    30
assignment_merge                          30
assignment_statistic_as

CalledProcessError: Command 'b'\n__conda_setup="$(\'/root/miniforge3/bin/conda\' \'shell.bash\' \'hook\' 2> /dev/null)"\nif [ $? -eq 0 ]; then\n    eval "$__conda_setup"\nelse\n    if [ -f "/root/miniforge3/etc/profile.d/conda.sh" ]; then\n        . "/root/miniforge3/etc/profile.d/conda.sh"\n    else\n        export PATH="/root/miniforge3/bin:$PATH"\n    fi\nfi\nunset __conda_setup\n\nif [ -f "/root/miniforge3/etc/profile.d/mamba.sh" ]; then\n    . "/root/miniforge3/etc/profile.d/mamba.sh"\nfi\n\ncd assoc_basic\nmamba activate snakemake\nsnakemake --sdm apptainer --snakefile ../MPRAsnakeflow/workflow/Snakefile \\\n  --configfile ../MPRAsnakeflow/resources/assoc_basic/config.yml \\\n  -c 1\n'' returned non-zero exit status 1.

In [ ]:
%%bash

__conda_setup="$('/root/miniforge3/bin/conda' 'shell.bash' 'hook' 2> /dev/null)"
if [ $? -eq 0 ]; then
    eval "$__conda_setup"
else
    if [ -f "/root/miniforge3/etc/profile.d/conda.sh" ]; then
        . "/root/miniforge3/etc/profile.d/conda.sh"
    else
        export PATH="/root/miniforge3/bin:$PATH"
    fi
fi
unset __conda_setup

if [ -f "/root/miniforge3/etc/profile.d/mamba.sh" ]; then
    . "/root/miniforge3/etc/profile.d/mamba.sh"
fi

cd assoc_basic
mamba activate snakemake
snakemake --sdm conda --snakefile ../MPRAsnakeflow/workflow/Snakefile \
  --configfile ../MPRAsnakeflow/resources/assoc_basic/config.yml \
  -c 1


Assuming unrestricted shared filesystem usage.
Building DAG of jobs...
Creating conda environment ../MPRAsnakeflow/workflow/envs/python3.yaml...
Environment for /content/MPRAsnakeflow/workflow/rules/../envs/python3.yaml created (location: .snakemake/conda/c50457acea2444acd8e75fe0a317f5fb_)
Creating conda environment ../MPRAsnakeflow/workflow/envs/bwa_samtools_picard_htslib.yaml...
Environment for /content/MPRAsnakeflow/workflow/rules/assignment/../../envs/bwa_samtools_picard_htslib.yaml created (location: .snakemake/conda/7b8791bda87729024a6bda910c75d51e_)
Creating conda environment ../MPRAsnakeflow/workflow/envs/default.yaml...
Environment for /content/MPRAsnakeflow/workflow/rules/../envs/default.yaml created (location: .snakemake/conda/6e6828c663b278a3ebf966a2b32525c3_)
Creating conda environment ../MPRAsnakeflow/workflow/envs/r.yaml...
Environment for /content/MPRAsnakeflow/workflow/rules/assignment/../../envs/r.yaml created (location: .snakemake/conda/4befc3cfe6b50ce710a2aef812330d

CalledProcessError: Command 'b'\n__conda_setup="$(\'/root/miniforge3/bin/conda\' \'shell.bash\' \'hook\' 2> /dev/null)"\nif [ $? -eq 0 ]; then\n    eval "$__conda_setup"\nelse\n    if [ -f "/root/miniforge3/etc/profile.d/conda.sh" ]; then\n        . "/root/miniforge3/etc/profile.d/conda.sh"\n    else\n        export PATH="/root/miniforge3/bin:$PATH"\n    fi\nfi\nunset __conda_setup\n\nif [ -f "/root/miniforge3/etc/profile.d/mamba.sh" ]; then\n    . "/root/miniforge3/etc/profile.d/mamba.sh"\nfi\n\ncd assoc_basic\nmamba activate snakemake\nsnakemake --sdm conda --snakefile ../MPRAsnakeflow/workflow/Snakefile \\\n  --configfile ../MPRAsnakeflow/resources/assoc_basic/config.yml \\\n  -c 1\n'' returned non-zero exit status 1.

In [ ]:
!ls -lh /content/assoc_basic/data/SRR10800986/

total 9.3G
-rw-r--r-- 1 root root 9.3G Jul 30 14:26 SRR10800986.sra
